# Experimentation only - Not worth it

In [4]:
import sys
!{sys.executable} -m pip install tqdm

You are using pip version 9.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import json
import numpy as np
import os
import pandas as pd
import urllib.request as ud 

In [43]:
url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1439014500&end=9999999999&period=300&resolution=auto'
openUrl = ud.urlopen(url)
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())
print(pd.DataFrame(d).shape)

(16732, 8)


In [44]:
df = pd.DataFrame(d)
#original_columns=[u'close', u'date', u'high', u'low', u'open']
#new_columns = ['Close','Timestamp','High','Low','Open']
#df = df.loc[:,original_columns]
#df.columns = new_columns
df.head()

,close,date,high,low,open,quoteVolume,volume,weightedAverage
0,273.879113,1439014500,273.879113,273.879113,273.879113,0.000000,0.000000,273.879113
1,281.000000,1439020800,281.000000,280.998470,280.998470,0.001137,0.319362,280.999707
2,282.235975,1439028000,282.235975,280.999910,280.999910,0.457676,128.997779,281.853770
3,263.407086,1439035200,277.303281,263.407086,277.303281,1.484353,396.969138,267.435719
4,262.647627,1439042400,262.647627,262.647627,262.647627,0.000194,0.051080,262.647627


In [45]:
#df.to_csv('data/bitcoin2015to2017.csv',index=None)

In [47]:
df['Datetime'] = pd.to_datetime(df['date'],unit='s')
df.head()

,close,date,high,low,open,quoteVolume,volume,weightedAverage,Datetime
0,273.879113,1439014500,273.879113,273.879113,273.879113,0.000000,0.000000,273.879113,2015-08-08 06:15:00
1,281.000000,1439020800,281.000000,280.998470,280.998470,0.001137,0.319362,280.999707,2015-08-08 08:00:00
2,282.235975,1439028000,282.235975,280.999910,280.999910,0.457676,128.997779,281.853770,2015-08-08 10:00:00
3,263.407086,1439035200,277.303281,263.407086,277.303281,1.484353,396.969138,267.435719,2015-08-08 12:00:00
4,262.647627,1439042400,262.647627,262.647627,262.647627,0.000194,0.051080,262.647627,2015-08-08 14:00:00


In [48]:
df.tail()

,close,date,high,low,open,quoteVolume,volume,weightedAverage,Datetime
16727,8577.000000,1559448000,8577.000000,8542.081860,8549.704175,4.310867,36882.826635,8555.779496,2019-06-02 04:00:00
16728,8707.108875,1559455200,8724.348409,8562.117240,8580.880061,95.427264,825609.894303,8651.719219,2019-06-02 06:00:00
16729,8698.062670,1559462400,8741.489160,8677.753284,8704.916795,61.668126,537309.806606,8712.925741,2019-06-02 08:00:00
16730,8720.000000,1559469600,8727.507501,8677.753284,8704.758663,14.688089,127811.956391,8701.741773,2019-06-02 10:00:00
16731,8770.000000,1559476800,8799.950000,8720.500000,8720.500000,20.114895,176238.799353,8761.606801,2019-06-02 12:00:00


In [50]:
prices= df.loc[:,'close'].values
times = df.loc[:,'Datetime'].values
prices.shape

(16732,)

In [56]:
input_step_size = 50
output_size = 30
sliding_window = False

outputs = []
inputs = []
output_times = []
input_times = []

if sliding_window:
    for i in range(len(prices)-input_step_size-output_size):
        inputs.append(prices[i:i + input_step_size])
        input_times.append(times[i:i + input_step_size])
        outputs.append(prices[i + input_step_size: i + input_step_size+ output_size])
        output_times.append(times[i + input_step_size: i + input_step_size+ output_size])
else:
    for i in range(0,len(prices)-input_step_size-output_size, input_step_size):
        inputs.append(prices[i:i + input_step_size])
        input_times.append(times[i:i + input_step_size])
        outputs.append(prices[i + input_step_size: i + input_step_size+ output_size])
        output_times.append(times[i + input_step_size: i + input_step_size+ output_size])

inputs= np.array(inputs)
outputs= np.array(outputs)
output_times = np.array(output_times)
input_times = np.array(input_times)

print(inputs.shape)
print(outputs.shape)
print(input_times.shape)
print(output_times.shape)

(334, 50)
(334, 30)
(334, 50)
(334, 30)


In [55]:
step_size = inputs.shape[0]
units= 50
second_units = 30
batch_size = 8
nb_features = inputs.shape[1]
epochs = 50
output_size=16
reg = 1
#output_file_name='bitcoin2015to2017_close_LSTM_1_tanh_leaky_areg_l1_'+ str(reg)
#split training validation
training_size = int(0.8* inputs.shape[0])
training_datas = inputs[:training_size,:]
training_labels = outputs[:training_size,:,0]
validation_datas = inputs[training_size:,:]
validation_labels = outputs[training_size:,:,0]


IndexError: too many indices for array